In [1]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy
import numpy as np
import gc
import shapely

from IPython.display import clear_output

from tqdm import tqdm
tqdm.pandas()


In [2]:
gemeenten = gpd.read_parquet("data/gemeenten_amenities.parquet")
# wijken = gpd.read_parquet("data/wijken.parquet")

In [3]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

def wk_total_amenities_entropy(gm_name, wijkarea, filter_i):
    L0_BLACKLIST, L1_BLACKLIST = ec.getfilter(filter_i)
    amenity_gdf = gpd.read_parquet(f"data/gm_amenities/amenities_{gm_name}.parquet")
    
    # filter out amenities not in the wijk
    amenity_gdf = amenity_gdf[amenity_gdf.within(wijkarea)]
    amenity_gdf.reset_index(drop=True, inplace=True)
    
    # apply filters
    amenity_gdf = amenity_gdf[~amenity_gdf.L0_category.isin(L0_BLACKLIST)]
    if L1_BLACKLIST:
        for key, value in L1_BLACKLIST.items():
            amenity_gdf = amenity_gdf[
                ~(
                    (amenity_gdf.L0_category == key)
                    & (amenity_gdf.L1_category.isin(value))
                )
            ]
    
    # total number of amenities
    total_amenities = len(amenity_gdf)
    
    points = [[point.x, point.y] for point in amenity_gdf.geometry]
    
    # calculate entropy
    L0 = amenity_gdf.loc[:, f"L0_category"].values
    L1 = amenity_gdf.loc[:, f"L1_category"].values
    # L0_entropy = _get_shannon_entropy(L0, base=2)
    # L1_entropy = _get_shannon_entropy(L1, base=2)
    try:
        L0_entropy = altieri_entropy(points, L0, base=2).entropy
    except:
        L0_entropy = 0
    try:
        L1_entropy = altieri_entropy(points, L1, base=2).entropy
    except:
        L1_entropy = 0
    
    del points, amenity_gdf, L0, L1
    gc.collect()
    
    return total_amenities, L0_entropy, L1_entropy

In [4]:
for filter in [1,2]:
    for part in [4]:
        wijken = gpd.read_parquet(f"data/wijken_parts/wijken_{part}a.parquet")

        for i, wijk in tqdm(wijken.iterrows(), total=len(wijken)):
            gm_name = wijk.gemeentenaam
            wijkarea = wijk.geometry
            total_amenities, L0_entropy, L1_entropy = wk_total_amenities_entropy(gm_name, wijkarea, filter)
            
            # wijken.at[i, f"total_amenities_{filter}"] = total_amenities
            wijken.at[i, f"L0_altieri_{filter}"] = L0_entropy
            wijken.at[i, f"L1_altieri_{filter}"] = L1_entropy
            
            del total_amenities, L0_entropy, L1_entropy
            gc.collect()

    wijken.to_parquet(f"data/wijken_parts/wijken_{part}a.parquet")

 35%|███▌      | 194/554 [01:17<02:18,  2.61it/s]/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:68: RuntimeWarning: invalid value encountered in divide
  pz = pz / pz.sum()
 35%|███▌      | 195/554 [01:18<02:14,  2.67it/s]/home/jleeuwenburgh/miniconda3/envs/urban/lib/python3.8/site-packages/spatialentropy/_altieri_entropy.py:63: RuntimeWarning: invalid value encountered in divide
  w = w / w.sum()
/

In [5]:
wijken.head(3)

,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,L1_shannon_1,total_amenities_2,L0_shannon_2,L1_shannon_2,L0_altieri_0,L1_altieri_0,L0_altieri_1,L1_altieri_1,L0_altieri_2,L1_altieri_2
2216,WK098803,Wijk 03 Molenakker en Kampershoek,GM0988,Weert,1.0,NEE,1093.0,3.0,1034.0,2935.0,...,2.271528,37.0,2.217474,2.271528,5.786247,5.979691,4.467683,4.587336,4.467683,4.587336
2217,WK098811,Wijk 11 Weert-Centrum,GM0988,Weert,1.0,NEE,2498.0,2.0,5091.0,4155.0,...,3.995719,127.0,2.489975,3.824953,5.882771,8.099642,4.984736,7.939001,5.045712,7.427118
2218,WK098812,Wijk 12 Biest,GM0988,Weert,1.0,NEE,1965.0,2.0,3740.0,2285.0,...,2.114685,36.0,1.989191,1.989191,5.565113,5.739388,4.081110,4.182401,3.941671,3.941671
